# 🌬️ NBeats-based Wind Power Forecasting (DK1)

Predict wind power output using historical windspeed and time features via N-Beats

In [ ]:
# Imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from darts import TimeSeries
from darts.models import NBEATSModel
from darts.dataprocessing.transformers import Scaler
from darts.metrics import r2_score
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from sklearn.metrics import mean_squared_error

In [ ]:
# Load your merged dataset
df = pd.read_csv("merged_training_data_2016.csv", parse_dates=["timestamp"])
df.rename(columns={"DK_1_wind_onshore_generation_actual": "wind_power_actual"}, inplace=True)

# Set timestamp as index
df = df.set_index("timestamp")

# Main target series
target_series = TimeSeries.from_series(df["wind_power_actual"])

# Covariates: meteorological features
covariates = df[[
    "windspeed_10m", "winddirection_10m", "temperature_2m",
    "surface_pressure", "relativehumidity_2m"
]]
covariate_series = TimeSeries.from_dataframe(covariates, fill_missing_dates=True, freq="H")

# Normalize
scaler_target = Scaler()
scaler_covariates = Scaler()

target_scaled = scaler_target.fit_transform(target_series)
covariates_scaled = scaler_covariates.fit_transform(covariate_series)

In [ ]:
# Forecast the last 7 days (168 hours)
train_target, val_target = target_scaled[:-168], target_scaled[-168:]
train_cov, val_cov = covariates_scaled[:-168], covariates_scaled[-168:]

model = NBEATSModel(
    input_chunk_length=168,   # past window
    output_chunk_length=24,   # predict 24h ahead
    n_epochs=100,
    batch_size=64,
    random_state=42,
    force_reset=True
)

# Fit model
model.fit(series=train_target, past_covariates=train_cov, verbose=True)

In [ ]:
# Prediction

val_cov = covariates_scaled[-336:]
pred = model.predict(n=168, past_covariates=val_cov)

val_target_aligned = val_target.slice(pred.start_time(), pred.end_time())

r2 = r2_score(val_target_aligned, pred)
rmse = np.sqrt(mean_squared_error(val_target_aligned.values(), pred.values()))

print(f"✅ Aligned R² Score: {r2:.4f}")
print(f"✅ Aligned RMSE: {rmse:.2f} MW")

pred = model.predict(n=168, past_covariates=val_cov)
pred.plot(label="forecast")
val_target.plot(label="actual")
plt.title("N-BEATS Wind Power Forecast (Last 7 Days)")
plt.legend()
plt.show()

In [ ]:
# Save model
model.save("nbeats_model.pth")

# Export predicted data
pred_pd = pred.pd_dataframe()
pred_pd.to_csv("nbeats_forecast.csv")